# Probabilistic Reasoning




# Full Joint Distributions

The students' data has been generated automatically, and is not realistic.
Let's first load the dataset and examine the first 10 rows.

In [1]:
import pandas as pd
import numpy as np

In [2]:
students = pd.read_csv("students.csv")
students.head(10)

,id,first_name,last_name,SCHOLARSHIP,STUDY,GPA
0,1,Belle,Travis,yes,BIT,7.0
1,2,Tammie,Longhi,yes,TCS,8.5
2,3,Cecily,Golborn,no,BIT,6.5
3,4,Vally,Abramchik,yes,TCS,7.5
4,5,Hardy,Garman,no,TCS,9.0
5,6,Harrietta,Doherty,yes,TCS,5.5
6,7,Cointon,Otton,no,BIT,9.0
7,8,Joellen,Linge,yes,BIT,9.5
8,9,Billi,Tadd,yes,TCS,8.5
9,10,Wallie,Alison,no,TCS,5.5


### 1.  
Calculating the following probability $$P(SCHOLARSHIP=yes)$$

In [37]:
(students.SCHOLARSHIP == 'yes').mean()

0.494

### 2. 
1.Calculating the following probability
$$P(SCHOLARSHIP=yes, GPA>7.5)$$

In [38]:
((students['SCHOLARSHIP']=='yes') & (students['GPA'] > 7.5 )).mean()

0.206

2. Calculating the probability $$P(GPA>7.5, SCHOLARSHIP=yes)$$
##### The values with 2.1 must be the same

In [39]:
((students['GPA'] > 7.5 ) & (students['SCHOLARSHIP']=='yes')).mean()

0.206

### 3.
Calculating the following probability
$$P(SCHOLARSHIP=yes, GPA>7.5,STUDY=BIT)$$

In [40]:
((students['SCHOLARSHIP']=='yes') & (students['GPA'] > 7.5 ) & (students['STUDY'] == 'BIT' )).mean()

0.104

# Your answer goes here

### 4.
Calculating the following conditional probability
$$P(SCHOLARSHIP=yes|GPA>7.5)$$

In [41]:
a = ((students['SCHOLARSHIP']=='yes') & (students['GPA'] > 7.5 )).mean()
b = (students['GPA'] > 7.5 ).mean()
a/b

0.5

### 5.
Calculating the same probability as in question 4 by using the Bayes rule.


First, we calculate $$P(GPA>7.5|SCHOLARSHIP=yes)$$ then use it to calculate 
$$P(SCHOLARSHIP=yes|GPA>7.5)$$

In [3]:
x = ((students['GPA'] > 7.5 ) & (students['SCHOLARSHIP']=='yes')).mean()
y = (students['SCHOLARSHIP']=='yes').mean()
prob_b_given_a = x/y
prob_a = (students.SCHOLARSHIP == 'yes').mean()
prob_b = (students.GPA > 7.5).mean()
prob_a_given_b = (prob_b_given_a*prob_a)/prob_b
print(prob_a_given_b)

0.5


### 6.
A vector of probabilities for the following expression $$P(SCHOLARSHIP|STUDY)$$

In [43]:
pd.crosstab(students.SCHOLARSHIP, students.STUDY, margins=True, normalize='columns')

STUDY,BIT,TCS,All
SCHOLARSHIP,,,
no,0.481013,0.528517,0.506
yes,0.518987,0.471483,0.494


### 7.
A vector of probabilities for the following expression $$ P (STUDY|SCHOLARSHIP)$$  

In [44]:
pd.crosstab(students.STUDY, students.SCHOLARSHIP, margins=True, normalize='columns')

SCHOLARSHIP,no,yes,All
STUDY,,,
BIT,0.450593,0.497976,0.474
TCS,0.549407,0.502024,0.526


### 8. 
Recreating full joint probability distribution, considering that the GPA variable is also binary, either GPA>7.5 or GPA<=7.5
The random variables we are interested in are SCHOLARSHIP, STUDY and GPA.

In [45]:
table = students.value_counts([students.SCHOLARSHIP, students.STUDY, students.GPA > 7.5], normalize=True)
print(table)
print(table.sum())

SCHOLARSHIP  STUDY  GPA  
no           TCS    False    0.160
yes          TCS    False    0.146
             BIT    False    0.142
no           BIT    False    0.140
             TCS    True     0.118
yes          BIT    True     0.104
             TCS    True     0.102
no           BIT    True     0.088
dtype: float64
1.0


## Balls and Urns

First, we represent how many urns we have and how many balls of each type they contain.

In [46]:
import numpy as np
from scipy.stats import binom

urns = np.array([[5,3],[3,5]])
numUrns = urns.shape[0]
priors = np.ones(numUrns) / numUrns

We draw balls with replacement. The first thing we do is to compute the probability of drawing $n$ white balls and $m$ blue balls, with replacement.

**1** We implement a function called "likelihood" which takes as parameters
1. The index of the urn
1. The number of white balls drawn
1. The number of blue balls drawn
    
The distribution for this kind of observation is called the Binomial distribution.

In [47]:
def _sum(arr):
    sum = 0
    for i in arr:
        sum = sum + i
 
    return(sum)

def likelihood(urnIndx, whitedrawn, blueDrawn):
    k = whitedrawn;
    n = whitedrawn + blueDrawn;
    urn = urns[urnIndx];
    p = urn[0]/_sum(urn);
    return(binom.pmf(k, n, p))

print(likelihood(1,3,2))

0.20599365234375


Now, we compute the posterior probability of having used the first urn given the observed number of white and blue balls. 

**2** We implement a function called "posterior" which takes the same parameters as the "likelihood" function. The implementation works for any number of urns.

We calculate:

What is the posterior probability of having picked urn 1 if we drew 3 white and 2 blue balls? What about 1 white and 3 blue balls?

In [48]:
# Your answer comes here
def pre_posterior(urnIndx, whitedrawn, blueDrawn):

    return(likelihood(urnIndx, whitedrawn, blueDrawn)*priors[urnIndx])

def posterior(urnIndx, whitedrawn, blueDrawn):
 
    evidence = 0
    values = range(numUrns)
    
    for i in values:
        evidence = evidence + pre_posterior(i, whitedrawn, blueDrawn)
    
    return(pre_posterior(urnIndx, whitedrawn, blueDrawn) / evidence)

print(posterior(0,3,2))
print(posterior(0,1,3))


0.6249999999999999
0.26470588235294107


## Entering college

We encode the probabilities of being smart, and an athlete and compute conditional probabilities.

In [49]:
pS = 0.4
pA = 0.1
pC_SA = np.array([[ 0.91, 0.9],[0.9, 0.04]])

**1** We implement a function that computes the joint probability of being smart (or not), entering college (or not) and being an athlete (or not). This functions takes three boolean parameters:
1. s, indicating whether the person is smart (True) or not Falss
1. c, indicating whether the person entered college
1. a, indicating whether the person is an athlete


In [50]:
def joint_probability(s, c, a):
    prob_c_given_a_s=pC_SA[s][a]
    prob_s=0.4
    prob_a=0.1
    joint_probability=prob_c_given_a_s*prob_a*prob_s
    return joint_probability

print(joint_probability(0,0,0))
    

0.03640000000000001


**2** We compute the probability that somebody is smart given that that person is an athlete, using marginalisation:

$$ p(s|a) = \frac{p(s,a)}{p(a)} $$

where

$$p(s,a) = p(s,c,a) + p(s,\neg c, a)$$

In [51]:
prob_c_given_a_s=pC_SA[0][0]
prob_not_c_given_a_s=1 - pC_SA[0][0]

prob_s_c_a = prob_c_given_a_s * 0.4 * 0.1
prob_s_not_c_a = prob_not_c_given_a_s* 0.4 * 0.1
prob_s_a = prob_s_c_a + prob_s_not_c_a

prob_s_given_a=prob_s_a/0.1

print(prob_s_given_a)

0.4000000000000001


**3** Next, we compute the probability that somebody is smart given that that person is in college

$$ P(s|c) = \frac{P(s,c)}{P(c)} = \frac{P(s,c,a) + P(s,c,\neg a)}{P(s,c,a)+P(s,c,\neg a)+P(\neg s,c,a) + P(\neg S,c,\neg a)}$$

In [52]:
s_c_a = pC_SA[0][0] * 0.4 * 0.1
s_c_not_a = pC_SA[0][1] * 0.4 * 0.9
not_s_c_a = pC_SA[1][0] * 0.6 * 0.1
not_s_c_not_a = pC_SA[1][1] * 0.6 * 0.9
s_given_c = (s_c_a + s_c_not_a) / (s_c_a + s_c_not_a + not_s_c_a + not_s_c_not_a)

print(s_given_c)

0.826605504587156


**4** Finally, we compute the probability that a person is smart, given that this person is in college and an athlete.

In [53]:
s_c_a=pC_SA[0][0] * 0.4 * 0.1
not_s_c_a = pC_SA[1][0] * 0.6 * 0.1
c_a= s_c_a + not_s_c_a

s_given_c_a= s_c_a/c_a

print(s_given_c_a)

0.40265486725663724


## Bayesian Networks

##### In the first part, we work with pyAgrum to create a simple Bayesian Network.

We start with the necessary imports

In [20]:
import pyAgrum as gum
import os
import pyAgrum.lib.notebook as gnb

Now we create a new Bayesian Network BN, called 'college', and store it in the variable bayesian_net.

In [21]:
bayesian_net=gum.BayesNet('college')
print(bayesian_net)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}


In this BN, we will have three random variables or nodes (S, A and C). We specify the possible value each random variable will take, in our case, all random variables will be set to either true or false (order matters here).

In [22]:
S=bayesian_net.add(gum.LabelizedVariable('S','Smart',["true","false"]))
A=bayesian_net.add(gum.LabelizedVariable('A','Athlete',["true","false"]))
C=bayesian_net.add(gum.LabelizedVariable('C','Collete',["true","false"]))

We now proceed to connect the nodes. Both nodes S and A are connected to C.

In [23]:
bayesian_net.addArc(S,C)
bayesian_net.addArc(A,C)


We can easily visualize our Bayesian Network to check whether its structure is correct.

In [24]:
bayesian_net

(pyAgrum.BayesNet<double>@0000024DE474B6A0) BN{nodes: 3, arcs: 2, domainSize: 8, dim: 6, mem: 96o}

We now proceed to fill the conditional probability tables for all our nodes.

In [25]:
bayesian_net.cpt(A).fillWith([0.1,0.9])

(pyAgrum.Potential<double>@0000024DE46ADE20) 
  A                |
true     |false    |
---------|---------|
 0.1000  | 0.9000  |

In [26]:
bayesian_net.cpt(S).fillWith([0.4,0.6])

(pyAgrum.Potential<double>@0000024DE46ADE80) 
  S                |
true     |false    |
---------|---------|
 0.4000  | 0.6000  |

In [27]:
bayesian_net.cpt(C).names

('C', 'S', 'A')

In [28]:
bayesian_net.cpt(C)[0,0,:] = [0.91, 0.09] # T=a1,A=new
bayesian_net.cpt(C)[0,1,:] = [0.9, 0.1] # T=a1,A=mid
bayesian_net.cpt(C)[1,0,:] = [0.9, 0.1] # T=a1,A=old
bayesian_net.cpt(C)[1,1,:] = [0.04, 0.96] # T=a2,A=new
bayesian_net.cpt(C)

(pyAgrum.Potential<double>@0000024DE46AE320) 
             ||  C                |
S     |A     ||true     |false    |
------|------||---------|---------|
true  |true  || 0.9100  | 0.0900  |
false |true  || 0.9000  | 0.1000  |
true  |false || 0.9000  | 0.1000  |
false |false || 0.0400  | 0.9600  |

Saving the model.

In [29]:
gum.saveBN(bayesian_net,os.path.join("./","college.bif"))

Now we proceed to make some inferences using our Bayesian network.

In [30]:
ie=gum.LazyPropagation(bayesian_net)

Calculating $P(s|a,c) and P(\neg s|a,c)$

In [31]:
ie.makeInference()
ie.setEvidence({'A':0, 'C':0})
ie.makeInference()
ie.posterior(S)

(pyAgrum.Potential<double>@0000024DE46AE280) 
  S                |
true     |false    |
---------|---------|
 0.4027  | 0.5973  |

Calculating $P(s|c) and P(\neg s|c)$

In [32]:
ie.makeInference()
ie.setEvidence({'C':0})
ie.makeInference()
ie.posterior(S)

(pyAgrum.Potential<double>@0000024DE46AE0E0) 
  S                |
true     |false    |
---------|---------|
 0.8266  | 0.1734  |

Calculating $P(s|a) and P(\neg s|a)$

In [33]:
ie.makeInference()
ie.setEvidence({'A':0})
ie.makeInference()
ie.posterior(S)

(pyAgrum.Potential<double>@0000024DE46AE0C0) 
  S                |
true     |false    |
---------|---------|
 0.4000  | 0.6000  |